# Lab 09 Examples (Karatsuba Fast Multiplication)

Click \<shift> \<enter> in each code cell to run the code. Be sure to start with the ```#include``` directives to load the required libraries.

In [ ]:
// For Lab 09, we are limited to using only <iostream>, <string>, and <cmath>

#include <iostream>
#include <string>
#include <cmath>

# Overview

In today's lab, we will compare the number of operations involved in addition and multiplication.
We will then look at the Divide and Conquer approach to multiplication because although the divide and conquer approach alone is not asymptotically faster than the elementary school method, it prepares us for the Karatsuba algorithm, which improves upon the divide and conquer method and is faster.

## The Number of Operations in Addition and Multiplication

When programming, we typically think of additional and multiplication as single operations that complete in constant time. However, this assumes that the numbers being added or multiplied fit within the fixed size of standard data types (like `int` or `float`). When working with very large numbers (e.g., thousands or millions of digits), it becomes important to consider how the number of operations scales as length $n$ grows.

Let $n$ be the number of digits (base 10) or bits (base 2) in the inputs. Then the running times are:

- Addition: $\mathcal{O}(n)$ — linear time.
- Multiplication (grade‑school): $\mathcal{O}(n^2)$ — quadratic time (each digit of one operand is multiplied by each digit of the other).

### Addition

```txt
  2518
+ 3841
  ----
  6359
```

The number of single-digit additions is linear to the number of digits $n$. We could iterate over the digits from right to left, and a single loop would execute $n$ times. Even if there is a carry, it only requires a constant amount of extra work per digit, so the loop doesn't have to repeat any extra times. Addition can be performed in a single pass over the length.

### Multiplication (elementary school)

```txt
      2518
    x 3841
    ------
      2518
    10072 
   20144  
   7554   
----------
   9671638
```

The number of single-digit multiplications is quadratic to the number of digits $n$. Each digit of one operand is multiplied by each digit of the other operand. If each operand has $n$ digits, there are $n \times n = n^2$ single-digit multiplications. Thus, the running time is $\mathcal{O}(n^2)$. If writing a program, nested loops would be used to multiply each digit of both operands.

Addition is also involved in multiplication, but because addition is asymptotically faster, the multiplication step dominates the overall running time, resulting in a total time complexity of $\mathcal{O}(n^2)$ instead of $\mathcal{O}(n^2 + n) = \mathcal{O}(n^2)$.

### A Number of Length 4

For numbers of length `4`, multiplication will require up to `16` single-digit multiplications ($4^2$). Addition will require up to `4` single-digit additions.  As $n$ increases, the difference becomes even more impactful.

![Linear v Quadratic](https://latessa.github.io/cpp-labs/images/Lab09/linear_v_quadratic.png)

## Divide and Conquer Multiplication

### An Alternate Representation of a Number

Note that another way of expressing a number such as `1984` is to split it in two parts:

$$1984 = (19 * 10^{\frac{n}{2}}) + (84 * 10^0) \mid n = \text{length of the complete number}$$

$$1900 + 84$$

The $10^2$ part shifts the `19` two places to the left (indicated by the 2 in the exponent).  The $10^0$ part shifts the `84` zero places to the left (meaning it stays in the same place) which is indicated by the 0 in the exponent.

This can also be done in binary. For example, the binary number `1101 1010` can be split into two parts:

$$1101\ 1010 = (1101 \texttt{ << } \frac{n}{2}) + (1010 \texttt{ << } 0) \mid n = \text{length of the complete number}$$

$$1101\ 0000 + 1010$$

We use this alternate representation of numbers to implement the divide and conquer multiplication so that we can split the original problem into smaller problems and then reassemble the results of the smaller problems to get the final result.

### Setting Up the Problem

Let's multiply two 4-digit numbers, `1984` times `2135`. Using our alternate representation, we can name each half of each number.  Let's refer to `1984` as `A` and `2135` as `B`. Let's refer to the left half of `A` as `AL` and the right half as `AR`. Similarly, let's refer to the left half of `B` as `BL` and the right half as `BR`.

```txt
    AL  BR
    19  84

    BL  BR
    21  35

    LENGTH n = 4
```

In base 10, we can express the multiplication of `A` and `B` as:

$$ (AL * 10^{\frac{n}{2}} + AR * 10^0) * (BL * 10^{\frac{n}{2}} + BR * 10^0) $$

Apply the distributive property (FOIL), we can rewrite the expression.  FOIL stands for First, Outside, Inside, Last. These are the four products we get when multiplying two binomials.

$$ (AL * BL)10^{n} + (AL * BR)10^{\frac{n}{2}} + (AR * BL)10^{\frac{n}{2}} + (AR * BR)10^{0} $$

$$ C_{LL} = (AL * BL) = (19 * 21) = 0399 $$
$$ C_{LR} = (AL * BR) = (19 * 35) = 0665 $$
$$ C_{RL} = (AR * BL) = (84 * 21) = 1764 $$
$$ C_{RR} = (AR * BR) = (84 * 35) = 2940 $$

Append the appropriate number of zeros (i.e., multiply by the appropriate power of 10) to each of these four products and then add them together to get the final result.  This is where bit shifts would be used in binary.

$$
\begin{aligned}
C_{LL}10^{n} &\;+\; & (C_{LR} + C_{RL})10^{\frac{n}{2}} &\;+\; & C_{RR}10^{0} \\
(0399)10^{4} &\;+\; & (0665 + 1764)10^{2} &\;+\; & (2940)10^{0} \\
3990000 &\;+\; & 242900 &\;+\; & 2940 \\
\end{aligned}
$$

$$ = 4,235,840 $$

Splitting the original numbers into halves, though, is often just the beginning. Two 1000-digit numbers split in halves are still two 500-digit numbers. Thanks to recursion, though, the rest of the work can be done with recursive calls to same function.

## The Recursive Pseudocode

```cpp
function multiply(A, B):
    if length(A) == 1 or length(B) == 1:
        return A * B  // base case: single-digit multiplication
    n = max(length(A), length(B))
    half_n = n / 2

    // Split A and B into left and right halves
    AL = high half of A
    AR = low half of A
    BL = high half of B
    BR = low half of B

    // 4 recursive calls to multiply the halves
    C_LL = multiply(AL, BL)
    C_LR = multiply(AL, BR)
    C_RL = multiply(AR, BL)
    C_RR = multiply(AR, BR) 

    // Combine the results and adjust by powers of 10
    return (C_LL * 10^n) + ((C_LR + C_RL) * 10^(n/2)) + C_RR
```